## Steam Reviews Get!

This will act as a live document as I use it to source, clean and analyze a dataset sourced from Steam. 


#### Objectives
* Source a dataset of all English reviews on steam!
    * Access publicly available api calls for store.steampowered
    * end with a dataframe for which a row will hold the app title/id, review author details, and review details
* Write and save dataset to Kaggle for convenience
* Thoroughly document process to help learn and also later reference as a tutorial

#### Concerns
* Avoiding DOS
    * I'm making an obscene number of requests from Valve what happens if I request too much?
    * Can I limit the amount of requests I make and request in a piecemeal fashion?
        * From research, it looks like steam limits the amount of requests allowed to 100,000 per calendar day. 
        * Approach two different ways: sleep an amount every request to ensure I don't go over (865ms per request) or count requests via a global variable and sleep on 100,000
        

In [67]:
import pandas as pd
import requests
import time

In [51]:


URL= "https://store.steampowered.com/appreviews/317710?json=1"
cursor = '*'
PARAMS = {
    'filter':'updated',
    'language':'english',
    'cursor': cursor,
    'purchase_type':'all',
    'num_per_page' : 1,
}

1533420
r = requests.get(URL, PARAMS)
r=r.json()

In [52]:
print(r)

print(r["query_summary"]['total_reviews'])
print(r["cursor"])

{'success': 1, 'query_summary': {'num_reviews': 1, 'review_score': 8, 'review_score_desc': 'Very Positive', 'total_positive': 346, 'total_negative': 50, 'total_reviews': 396}, 'reviews': [{'recommendationid': '115975941', 'author': {'steamid': '76561198091142053', 'num_games_owned': 239, 'num_reviews': 2, 'playtime_forever': 6438, 'playtime_last_two_weeks': 9, 'playtime_at_review': 6428, 'last_played': 1655940748}, 'language': 'english', 'review': "If you like anything you see of this, or you go and play the Demo version and like what you see, the whole game is exactly as shown. Fast paced, quick to get started (as in you die and can, within a second, start again), and all about memorizing the different segments of each level. Not a forgiving game, and if you have anger issues, be prepared to step back and breath. But really rewarding when you finally Beat it and get to level 9 and not just survive, but flourish.\n\nWarning, How the game plays it directly related to your frame rate. Ev

In [94]:
# print(r["reviews"])
df = pd.json_normalize(r["reviews"])
# df.head()

In [68]:
def getAppReviews(app, step, pause):
    URL = "https://store.steampowered.com/appreviews/" + str(app) +"?json=1"

    tally=step
    PARAMS = {
        'language':'english',
        'filter':'updated',
        'purchase_type':'all',
        'cursor': '*', #initial value defined in steam's documentation
        'num_per_page' : step,
    }

    r = requests.get(URL, PARAMS)
    r=r.json()
    nCursor = {'cursor' : r["cursor"]}
    PARAMS.update(nCursor)
    total = r["query_summary"]['total_reviews']
    df = pd.json_normalize(r["reviews"])
    while tally < total:
        time.sleep(pause)
        r = requests.get(URL, PARAMS)
        r=r.json()
        df = pd.concat([df, (pd.json_normalize(r["reviews"]))])
        tally += step
        nCursor = {'cursor' : r["cursor"]}
        PARAMS.update(nCursor)
    return df
    

In [70]:
df = getAppReviews(317710, 51, 0.87)

In [65]:
df.shape

(396, 20)

We now have a method to get reviews given a specific app. Next we get a list of all apps to call from.

In [6]:
apps = requests.get("https://api.steampowered.com/ISteamApps/GetAppList/v2/")
apps=apps.json()

AttributeError: 'dict' object has no attribute 'head'

In [13]:
df = pd.json_normalize(apps["applist"]["apps"])

In [14]:
df.head

<bound method NDFrame.head of           appid                                         name
0       2016512                                             
1       2025850              X Wars Deluxe - Line Effect DLC
2       2025960                                         海岛守卫
3       2025980  FPS Seduce - Hentai - Appreciation Room DLC
4       2026000                         Our Adventurer Guild
...         ...                                          ...
146177   838410                          Tabletop Playground
146178   359550               Tom Clancy's Rainbow Six Siege
146179  1990990                                  AstroTrucks
146180   473950                              Manifold Garden
146181  1012790                           Into the Radius VR

[146182 rows x 2 columns]>

In [16]:
dc=df.dropna()
dc.head

<bound method NDFrame.head of           appid                                         name
0       2016512                                             
1       2025850              X Wars Deluxe - Line Effect DLC
2       2025960                                         海岛守卫
3       2025980  FPS Seduce - Hentai - Appreciation Room DLC
4       2026000                         Our Adventurer Guild
...         ...                                          ...
146177   838410                          Tabletop Playground
146178   359550               Tom Clancy's Rainbow Six Siege
146179  1990990                                  AstroTrucks
146180   473950                              Manifold Garden
146181  1012790                           Into the Radius VR

[146182 rows x 2 columns]>